In [1]:
import os
import sys
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from scipy.special import expit

import import_ipynb
from part1_data_fitting import Model

importing Jupyter notebook from part1_data_fitting.ipynb
The source folder is: /Users/aureliennioche/Documents/PythonProjects/ProspecTonk/data/source
The figure folder is: /Users/aureliennioche/Documents/PythonProjects/ProspecTonk/fig
The backup folder is: /Users/aureliennioche/Documents/PythonProjects/ProspecTonk/data/backup
best_param [ 0.72158801  0.72756233 -0.20691704]
  4%|▎         | 4/113 [01:30<41:10, 22.67s/it]


KeyboardInterrupt: 

In [ ]:
FIG_FOLDER = 'fig'
SOURCE_FOLDER = os.path.join('data', 'source')
BACKUP_FOLDER = os.path.join('data', 'backup')
print(f"The source folder is: {os.path.abspath(SOURCE_FOLDER)}")
print(f"The figure folder is: {os.path.abspath(FIG_FOLDER)}")
print(f"The backup folder is: {os.path.abspath(BACKUP_FOLDER)}")

# Create folders
for f in SOURCE_FOLDER, FIG_FOLDER, BACKUP_FOLDER:
    os.makedirs(f, exist_ok=True)

### Load the results

In [ ]:
df_fit = pd.read_csv(os.path.join(BACKUP_FOLDER, "df_fit.csv"))
df_fit.date = pd.to_datetime(df_fit.date)
df_fit

In [ ]:
df_elo = pd.read_csv(os.path.join(BACKUP_FOLDER, "df_elo.csv"), index_col=0)
df_elo.index = pd.to_datetime(df_elo.index)
df_elo

# Correlation

In [ ]:
monkeys = df_fit.monkey.unique()
cond = ("gain", "loss")

for i, m in enumerate(monkeys):
    
    fig, axes = plt.subplots(figsize=(10, 3*n_param), nrows=n_param, ncols=len(cond))
    fig.suptitle(m, fontsize=20)
    
    for j, cd in enumerate(cond):
        
        df_p = df_fit[(df_fit.monkey == m) & (df_fit.condition == cd)]
        
        df_elo_m = df_elo[m.lower()]
        df_elo_m = df_elo_m[df_elo_m.index >= min(df_p.date)]
        
        for k in range(n_param):
            
            param = Model.param_labels[k]
            
            df_ = pd.DataFrame({
                param: df_p[param].values, 
                "elo": df_elo_m        
            }, index=df_p.date)
            
            r_, p = scipy.stats.pearsonr(df_.elo, df_[param].values)
            
            ax = axes[k, j]
            t = f"{cd}; r={r_:.2f}; p={p:.3f}"
            ax.set_title(t)
            
            sns.regplot(x="elo", y=param, data=df_, color=colors[k], ax=ax)
    
    plt.tight_layout()
    plt.show()

In [ ]:
df_fit_day = pd.read_csv(os.path.join(BACKUP_FOLDER, "df_fit_day.csv"))
df_fit_day.date = pd.to_datetime(df_fit_day.date)
df_fit_day

# Regression

In [ ]:
monkeys = df_fit.monkey.unique()
cond = ("gain", "loss")

for i, m in enumerate(monkeys):
    
    fig, axes = plt.subplots(figsize=(10, 3*n_param), nrows=n_param, ncols=len(cond))
    fig.suptitle(m, fontsize=20)
    
    for j, cd in enumerate(cond):
        
        df_p = df_fit[(df_fit.monkey == m) & (df_fit.condition == cd)]
        
        df_elo_m = df_elo[m.lower()]
        df_elo_m = df_elo_m[df_elo_m.index >= min(df_p.date)]
        
        for k in range(n_param):
            
            param = Model.param_labels[k]
            
            df_ = pd.DataFrame({
                param: df_p[param].values, 
                "elo": df_elo_m        
            }, index=df_p.date)
            
            r_, p = scipy.stats.pearsonr(df_.elo, df_[param].values)
            
            ax = axes[k, j]
            t = f"{cd}; r={r_:.2f}; p={p:.3f}"
            ax.set_title(t)
            
            sns.regplot(x="elo", y=param, data=df_, color=colors[k], ax=ax)
    
    plt.tight_layout()
    plt.show()

In [ ]:
monkeys = df_fit.monkey.unique()
cond = df_fit.condition.unique()
param = Model.param_labels    

d = [] 

for i, m in enumerate(monkeys):
    
    for j, cd in enumerate(cond):
        
        df_p = df_fit[(df_fit.monkey == m) & (df_fit.condition == cd)]
        
        df_elo_m = df_elo[m.lower()]
        df_elo_m = df_elo_m[df_elo_m.index >= min(df_p.date)]
        
        for k, pr in enumerate(param):
  
            df_ = pd.DataFrame({
                pr: df_p[pr].values, 
                "elo": df_elo_m        
            }, index=df_p.date)
            
            r_, p = scipy.stats.pearsonr(df_.elo, df_[pr].values)
            
            d.append({"monkey": m, "condition": cd, "param": pr, "r": r_, "p": p})

df = pd.DataFrame(d)
            
for cd in cond:
    for pr in param:
        d = df[(df.param == pr) & (df.condition == cd)]
        sgn = d.p < 0.05
        pos = d[sgn].r > 0
        neg = d[sgn].r < 0
        print(f"{pr}-{cd}: {np.sum(sgn)} sign.; ({np.sum(pos)} +, {np.sum(neg)} -)")